<a href="https://colab.research.google.com/github/Voldemort-vk/WIDS---2025/blob/main/Pipeline_using_Hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets tokenizers torch

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("imdb")
print(dataset)

In [ ]:
import re

def Preprocessing(Review_set):
  Review_set["text"] = re.sub(r"<.*?>", "", Review_set["text"])  #REMOVE HTML TAGS
  return Review_set

dataset = dataset.map(Preprocessing)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_dataset = dataset.map(tokenize, batched=True)

In [ ]:
!pip install --upgrade transformers

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  _, _, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["test"], compute_metrics=compute_metrics)
trainer.train()

In [ ]:
result = model.evaluate()
print(result)

In [ ]:
model.save_pretrained("./fine_tuned_model")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

loaded_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")
loaded_model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)
loaded_model.eval()

review = "The movie was absolutely beautiful:)"

tokenized_review = loaded_tokenizer([review], padding="max_length", truncation=True)

input_ids = tokenized_review["input_ids"].to(device)
attention_mask = tokenized_review["attention_mask"].to(device)

with torch.no_grad():
    outputs = loaded_model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    pred = logits.argmax(-1).item()

label_map = {0: "Negative", 1: "Positive"}
print(label_map[pred])
